# Preprocesado de Noticias

Importar todas las librerías necesarias

In [1]:
import pandas as pd
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import os
import re
import html
import warnings

Definir directorios de donde queremos cargar nuestras noticias y donde queremos guardarlas una vez preprocesadas

In [2]:

# Usa el directorio actual del notebook como raíz del proyecto
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define las rutas a los directorios de datos (raw y processed)
RAW_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'raw')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'processed')

# Crea el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Lista todos los archivos .parquet en el directorio RAW_DIR
parquet_files = [f for f in os.listdir(RAW_DIR) if f.endswith('.parquet')]

# Muestra el número total de archivos encontrados
print(f"Se han encontrado {len(parquet_files)} archivos parquet para procesar.\n")

# Lista los nombres de los archivos encontrados
for file in parquet_files:
    print(file)

Se han encontrado 7 archivos parquet para procesar.

amazon_news.parquet
apple_news.parquet
google_news.parquet
meta_news.parquet
microsoft_news.parquet
nvidia_news.parquet
tesla_news.parquet


Procesamiento de los archivos parquet: en esta sección se limpia y transforma el contenido de cada noticia, eliminando HTML (utilizando BeautifulSoup), normalizando saltos de línea y espacios, para generar una versión lista para la tarea de summarization.

In [3]:
# Ignorar warnings de BeautifulSoup sobre URLs
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Procesar cada archivo
for filename in parquet_files:
    print(f"\nProcesando: {filename}")
    
    input_path = os.path.join(RAW_DIR, filename)
    df = pd.read_parquet(input_path)
    
    # Eliminar las filas con valores nulos en la columna body y teaser
    df = df.dropna(subset=["body", "teaser"], how="all").reset_index(drop=True)
    
    # Limpiar HTML y extraer texto
    df["clean_body"] = df["body"].apply(
        lambda x: BeautifulSoup(x, "html.parser").get_text(separator="\n", strip=True)
    )
    
    # Convertir entidades HTML a caracteres normales
    df["clean_body"] = df["clean_body"].apply(html.unescape)
    
    # Reemplazar saltos de línea pegados a palabras por un espacio
    df["clean_body"] = df["clean_body"].apply(lambda x: re.sub(r'(?<=\S)\n(?=\S)', ' ', x))
    
    # Reducir saltos de línea múltiples a uno solo
    df["clean_body"] = df["clean_body"].apply(lambda x: re.sub(r'\n+', '\n', x))
    
    # Limpiar espacios sobrantes al inicio/final
    df["clean_body"] = df["clean_body"].str.strip()
    
    # Eliminar la columna original body
    df = df.drop(columns=["body"])
    
    # Guardar con el mismo nombre de archivo
    output_path = os.path.join(OUTPUT_DIR, filename)
    df.to_parquet(output_path, engine="pyarrow", index=False)
    print(f"Guardado: {output_path} ({len(df)} filas)")

print("\nTodos los archivos han sido procesados!")
print("Archivos guardados en:", OUTPUT_DIR)


Procesando: amazon_news.parquet
Guardado: /home/adiez/Desktop/News2Stock/data/news/processed/amazon_news.parquet (12868 filas)

Procesando: apple_news.parquet
Guardado: /home/adiez/Desktop/News2Stock/data/news/processed/apple_news.parquet (24766 filas)

Procesando: google_news.parquet
Guardado: /home/adiez/Desktop/News2Stock/data/news/processed/google_news.parquet (13773 filas)

Procesando: meta_news.parquet
Guardado: /home/adiez/Desktop/News2Stock/data/news/processed/meta_news.parquet (15938 filas)

Procesando: microsoft_news.parquet
Guardado: /home/adiez/Desktop/News2Stock/data/news/processed/microsoft_news.parquet (13206 filas)

Procesando: nvidia_news.parquet
Guardado: /home/adiez/Desktop/News2Stock/data/news/processed/nvidia_news.parquet (10308 filas)

Procesando: tesla_news.parquet
Guardado: /home/adiez/Desktop/News2Stock/data/news/processed/tesla_news.parquet (27796 filas)

Todos los archivos han sido procesados!
Archivos guardados en: /home/adiez/Desktop/News2Stock/data/news/p